In [1]:
import trimesh
from trimesh import transformations as trf
import numpy as np
import time
import os
from tqdm import tqdm
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline

### Ray test method to vectorize mesh models

In [2]:
#for rotation
def trig(angle):
    r = np.deg2rad(angle)
    return r

def rotation_matrix(alpha=-90, beta=0, gamma=0):
    #rotate the mesh
    alpha = trig(alpha)
    beta = trig(beta)
    gamma = trig(gamma)

    origin, xaxis, yaxis, zaxis = [0, 0, 0], [1, 0, 0], [0, 1, 0], [0, 0, 1]

    Rx = trf.rotation_matrix(alpha, xaxis)
    Ry = trf.rotation_matrix(beta, yaxis)
    Rz = trf.rotation_matrix(gamma, zaxis)

    R = trf.concatenate_matrices(Rx, Ry, Rz)           
    R2=R[:3,:3]

    return R2

def write_csv_file(file_name, folder_name_list, vector_list):
    # Open a new CSV file for writing
    with open(file_name, 'w') as f:
        # Loop over the file_data list and write each row to the CSV file
        if len(folder_name_list) != len(vector_list):
            for i in range(len(vector_list)):
                file_name = folder_name_list[i]
                data = vector_list[i]
                # Write the file name to the first column
                f.write(file_name + ',')
                # Write the NumPy array to the remaining columns
                np.savetxt(f, data[np.newaxis], delimiter=',')

def visualize_mesh_point_matrix(mesh, points):
     #visualization
    # Plot the trimesh object
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.plot_trisurf(mesh.vertices[:, 0], mesh.vertices[:, 1], mesh.vertices[:, 2], triangles=mesh.faces, color='red')

    # Plot the point grid
    ax.scatter(points[:, 0], points[:, 1], points[:, 2], color='blue', alpha=0.5)

    # Set the axis limits
    ax.set_xlim(-3, 3)
    ax.set_ylim(-2, 2)
    ax.set_zlim(-2, 2)

    # Show the plot
    plt.show()


In [3]:
#Change the vectorization method to mesh.contains(points), 
#no need to calculate the sdf distance

def ray_test_vectorization(mesh, x=50, y=20, z=20):
    '''
    calculate binary representation vector of a car shape
       input: mesh
       output: vector representation of a mesh
       [35, 12, 12] = 5040
       [40, 16, 16] = 10240
       [50, 20, 20] = 20000
       '''
    
    start_time = time.time()
    
    #original mass center and bounding box
    # print(mesh.center_mass)
    # bbx_ex = mesh.bounding_box.primitive.extents
    # print(bbx_ex)#bounding box size

    R2 = rotation_matrix(alpha=-90, beta=0, gamma=0)
    # The rotation matrix is applyed to the mesh
    mesh.vertices = np.matmul(mesh.vertices, R2)
    mesh.vertices -= mesh.center_mass
    # print(mesh.center_mass)
    mesh.vertices = mesh.vertices * 5 #scale to 5*2*2 box
    # bbx_ex = mesh.bounding_box.primitive.extents
    # print(bbx_ex) #bounding box size
    
    #construct points matrix 
    x_p = np.linspace(-2.5, 2.5, x) 
    y_p = np.linspace(-1.0, 1.0, y)
    z_p = np.linspace(-1.0, 1.0, z)
    points = np.vstack(np.meshgrid(x_p, y_p, z_p)).reshape(3,-1).T
    
    #if a point inside the mesh
    sd = mesh.contains(points)
    del mesh
    sd = sd * 1 #convert boolean to int
    
    #show execution time
    execution_time = time.time() - start_time
   #  print("execution time: %s" %(execution_time))
    
   #  #visualize the mesh and point grid
   #  visualize_mesh_point_matrix(mesh, points)
    
    return sd, execution_time

In [4]:
settings =  [[25, 10, 10]] #[[35, 12, 12], [40, 16, 16], [50, 20, 20]] 
dimensions = ['2500'] # ['5040', '10240', '20000']
folders = [f for f in os.listdir('./cars')]

In [5]:
for i, setting in enumerate(settings):
    dim = dimensions[i]
    vec_list = []
    total_time = 0
    x_pm, y_pm, z_pm = setting
    for folder in tqdm(folders[:]):
        mesh_file = os.path.join('./cars' + '/' + folder + '/' + 'car_reg.obj')
        mesh = trimesh.load(mesh_file)
        # print(mesh.is_watertight)
        if mesh.is_watertight:
            sd, exe_time = ray_test_vectorization(mesh, x=x_pm, y=y_pm, z=z_pm)
            vec_list.append(np.array(sd))
            total_time += exe_time
        else: 
            print(folder)
    ave_time = total_time / len(folders)
    print(f"Average time for processing one mesh: {ave_time}")
    write_csv_file(f'./{dim}_vectors.csv', folders, vec_list)

  1%|▏         | 6/439 [00:00<00:18, 23.62it/s]

True
True
True
True
True
True


  3%|▎         | 12/439 [00:00<00:17, 24.81it/s]

True
True
True
True
True
True


  3%|▎         | 15/439 [00:00<00:16, 25.17it/s]

...

 64%|██████▎   | 279/439 [00:10<00:05, 26.76it/s]

True
True
True
True
True
True


 65%|██████▍   | 284/439 [00:11<00:06, 25.70it/s]

False
True
True
True
True


KeyboardInterrupt: 